# **Leetcode Problem Recommendation System**

## **Data Pre-Processing**

In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np

data = json.loads(open("data.json", "r").read())
new_data = defaultdict(list)
for key in data.keys():
    for attribute in data[key].keys():
        new_data[attribute].append(data[key][attribute])

with open("updated_data.json", 'w+') as f:
    f.write(json.dumps(new_data))

df = pd.DataFrame(new_data)
df.head()

,accuracy,title,titleSlug,topics,difficulty,questionId,questionFrontendId,question,link,likes,dislikes,likability
0,54.199137,Two Sum,two-sum,"[array, hash-table]",easy,1,1,<p>Given an array of integers <code>nums</code...,https://leetcode.com/problems/two-sum,58795,2097,96.556198
1,44.566495,Add Two Numbers,add-two-numbers,"[linked-list, math, recursion]",medium,2,2,<p>You are given two <strong>non-empty</strong...,https://leetcode.com/problems/add-two-numbers,32082,6445,83.271472
2,35.730692,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,"[hash-table, string, sliding-window]",medium,3,3,"<p>Given a string <code>s</code>, find the len...",https://leetcode.com/problems/longest-substrin...,40582,1956,95.401758
3,41.961454,Median of Two Sorted Arrays,median-of-two-sorted-arrays,"[array, binary-search, divide-and-conquer]",hard,4,4,<p>Given two sorted arrays <code>nums1</code> ...,https://leetcode.com/problems/median-of-two-so...,28998,3261,89.891193
4,34.741408,Longest Palindromic Substring,longest-palindromic-substring,"[two-pointers, string, dynamic-programming]",medium,5,5,"<p>Given a string <code>s</code>, return <em>t...",https://leetcode.com/problems/longest-palindro...,29884,1837,94.208884


In [2]:
difficulty_map = {"easy" : 1, "medium" : 2, "hard" : 3}

df["difficulty"] = df["difficulty"].map(difficulty_map)
df.head()

,accuracy,title,titleSlug,topics,difficulty,questionId,questionFrontendId,question,link,likes,dislikes,likability
0,54.199137,Two Sum,two-sum,"[array, hash-table]",1,1,1,<p>Given an array of integers <code>nums</code...,https://leetcode.com/problems/two-sum,58795,2097,96.556198
1,44.566495,Add Two Numbers,add-two-numbers,"[linked-list, math, recursion]",2,2,2,<p>You are given two <strong>non-empty</strong...,https://leetcode.com/problems/add-two-numbers,32082,6445,83.271472
2,35.730692,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,"[hash-table, string, sliding-window]",2,3,3,"<p>Given a string <code>s</code>, find the len...",https://leetcode.com/problems/longest-substrin...,40582,1956,95.401758
3,41.961454,Median of Two Sorted Arrays,median-of-two-sorted-arrays,"[array, binary-search, divide-and-conquer]",3,4,4,<p>Given two sorted arrays <code>nums1</code> ...,https://leetcode.com/problems/median-of-two-so...,28998,3261,89.891193
4,34.741408,Longest Palindromic Substring,longest-palindromic-substring,"[two-pointers, string, dynamic-programming]",2,5,5,"<p>Given a string <code>s</code>, return <em>t...",https://leetcode.com/problems/longest-palindro...,29884,1837,94.208884


In [3]:
df.isna().sum()

accuracy                0
title                   0
titleSlug               0
topics                  0
difficulty              0
questionId              0
questionFrontendId      0
question              685
link                    0
likes                   0
dislikes                0
likability              0
dtype: int64

In [4]:
df.dropna(inplace=True)
df.count()

accuracy              2668
title                 2668
titleSlug             2668
topics                2668
difficulty            2668
questionId            2668
questionFrontendId    2668
question              2668
link                  2668
likes                 2668
dislikes              2668
likability            2668
dtype: int64

In [5]:
# Reset index to avoid KeyErrors
df.reset_index(drop=True, inplace=True)
df.to_json("updated_data.json")

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

# User solved questions
solved_slugs = ['two-sum']

# 1. Sentence embeddings for questions
model = SentenceTransformer('all-mpnet-base-v2')
df['embedding'] = df['question'].apply(lambda q: model.encode(q, show_progress_bar=False))

# Normalize features
scaler = MinMaxScaler()
df[['difficulty', 'likability', 'accuracy']] = scaler.fit_transform(df[['difficulty', 'likability', 'accuracy']])

# 2. Build the MRF graph
def build_mrf(df):
    G = nx.Graph()
    
    for _, row in df.iterrows():
        node = row['titleSlug']
        G.add_node(node, potential=1.0, embedding=row['embedding'])
    
    for i, row_i in df.iterrows():
        for j, row_j in df.iterrows():
            if i >= j:
                continue
            
            # Edge weight: combination of cosine similarity and topic overlap
            sim_score = cosine_similarity([row_i['embedding']], [row_j['embedding']])[0][0]
            topic_overlap = len(set(row_i['topics']) & set(row_j['topics'])) / len(set(row_i['topics']) | set(row_j['topics']))
            edge_weight = 0.7 * sim_score + 0.3 * topic_overlap
            
            G.add_edge(row_i['titleSlug'], row_j['titleSlug'], weight=edge_weight)
    
    return G

graph = build_mrf(df)

# 3. Belief Propagation with Damping
def belief_propagation(graph, solved_slugs, max_iters=10, damping=0.5):
    beliefs = {node: graph.nodes[node]['potential'] for node in graph.nodes}
    previous_beliefs = beliefs.copy()
    
    for _ in range(max_iters):
        for node in graph.nodes:
            if node in solved_slugs:
                continue
            
            message = 1.0
            for neighbor in graph.neighbors(node):
                edge_weight = graph[node][neighbor]['weight']
                message *= edge_weight * previous_beliefs[neighbor]
            
            new_belief = graph.nodes[node]['potential'] * message
            beliefs[node] = damping * new_belief + (1 - damping) * previous_beliefs[node]
        
        previous_beliefs = beliefs.copy()
    
    # Normalize beliefs
    total_belief = sum(beliefs.values())
    for node in beliefs:
        beliefs[node] /= total_belief
    
    return beliefs

beliefs = belief_propagation(graph, solved_slugs)

# 4. Diversify recommendations using MMR
def get_recommendations(beliefs, graph, solved_slugs, top_k=5, lambda_div=0.7):
    ranked_nodes = sorted(beliefs.items(), key=lambda x: x[1], reverse=True)
    recommendations = []
    
    while len(ranked_nodes) > 0 and len(recommendations) < top_k:
        candidate = ranked_nodes.pop(0)
        if candidate[0] in solved_slugs:
            continue
        
        # Calculate MMR
        mmr_score = lambda_div * candidate[1] - (1 - lambda_div) * max(
            [graph[candidate[0]][rec]['weight'] for rec in recommendations] or [0]
        )
        
        recommendations.append((candidate[0], mmr_score))
    
    return [rec[0] for rec in sorted(recommendations, key=lambda x: x[1], reverse=True)]

recommendations = get_recommendations(beliefs, graph, solved_slugs, top_k=5)

# 5. Print recommendations
print("Recommended Questions:")
for rec in recommendations:
    print(df[df['titleSlug'] == rec]['titleSlug'].values[0])

Recommended Questions: [('find-first-and-last-position-of-element-in-sorted-array', 0.8010920062005068), ('4sum', 0.7975759205069344), ('search-insert-position', 0.795626880116056), ('binary-search', 0.7523763161182005), ('3sum-closest', 0.7429428175506935)]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

DF = 
# Load DataFrame
def load_data(file_path="updated_data.json"):
    df = pd.read_json(file_path)
    df = preprocess_data(df)
    return df

# Function to preprocess data
def preprocess_data(df):
    # Normalize 'likability' and 'accuracy'
    scaler = MinMaxScaler()
    df[['likability', 'accuracy']] = scaler.fit_transform(df[['likability', 'accuracy']])
    return df

# Function to calculate the joint probability of accuracy and difficulty
def calculate_potential_matrix(df):
    joint_prob = pd.crosstab(df['accuracy'], df['difficulty'], normalize='all')
    
    # We will use the joint probability distribution to form the potential matrix
    potential_matrix = np.zeros((2, 3))  # 2 values for accuracy (0, 1) and 3 values for difficulty (1, 2, 3)
    
    # Fill the potential matrix based on the joint distribution
    for acc in range(2):  # Accuracy can be 0 or 1
        for diff in range(1, 4):  # Difficulty can be 1, 2, or 3
            if diff in joint_prob.columns:
                potential_matrix[acc, diff-1] = joint_prob.loc[acc, diff] if acc in joint_prob.index else 0
    
    return potential_matrix

# Markov Random Field (MRF) Implementation
class MarkovRandomField:
    def __init__(self):
        self.edges = {}
        self.nodes = {}
        self.potentials = {}
    
    def add_edge(self, node1, node2, potential):
        if node1 not in self.nodes:
            self.nodes[node1] = {}
        if node2 not in self.nodes:
            self.nodes[node2] = {}
        
        # Store the potential in the 'potentials' dictionary
        self.potentials[(node1, node2)] = potential
    
    def compute_potential(self, node1, node2, val1, val2):
        # Access the potential matrix for the edge (node1, node2)
        return self.potentials[(node1, node2)][val1, val2]

# Belief Propagation Implementation
def belief_propagation(mrf, max_iter=10):
    messages = {}
    
    # Initialize messages to ones
    for (node1, node2) in mrf.edges:
        messages[(node1, node2)] = np.ones(2)  # Uniform initialization for binary values (0, 1)
    
    # Perform message updates
    for _ in range(max_iter):
        for (node1, node2) in mrf.edges:
            # We assume that nodes are binary (0, 1) for 'accuracy' and categorical for 'difficulty'
            incoming_messages = np.prod([messages[(nbr, node1)] for nbr in mrf.nodes[node1] if nbr != node2], axis=0)
            
            # We need to calculate messages correctly: values for each node (0 or 1 for accuracy)
            for val1 in range(2):  # For accuracy, which can take values 0 or 1
                for val2 in range(3):  # For difficulty, which can take values 1, 2, or 3
                    # Update the message with the correct potential value
                    messages[(node1, node2)] = mrf.compute_potential(node1, node2, val1, val2)
    
    return messages

# Custom Topic Modeling (similar to LDA)
def custom_topic_model(corpus, n_topics=2, alpha=0.1, beta=0.01):
    vocab = {word: idx for idx, word in enumerate(set(' '.join(corpus).split()))}
    word_topic_matrix = np.zeros((len(vocab), n_topics))
    doc_topic_matrix = np.zeros((len(corpus), n_topics))
    
    doc_word_topic = []
    for i, doc in enumerate(corpus):
        topics = []
        for word in doc.split():
            topic = np.random.randint(0, n_topics)
            topics.append(topic)
            word_topic_matrix[vocab[word], topic] += 1
            doc_topic_matrix[i, topic] += 1
        doc_word_topic.append(topics)
    
    for _ in range(50):
        for d, doc in enumerate(corpus):
            for i, word in enumerate(doc.split()):
                topic = doc_word_topic[d][i]
                word_topic_matrix[vocab[word], topic] -= 1
                doc_topic_matrix[d, topic] -= 1
                
                topic_dist = (word_topic_matrix[vocab[word], :] + beta) * (doc_topic_matrix[d, :] + alpha)
                new_topic = np.argmax(topic_dist / topic_dist.sum())
                
                doc_word_topic[d][i] = new_topic
                word_topic_matrix[vocab[word], new_topic] += 1
                doc_topic_matrix[d, new_topic] += 1
    
    return doc_topic_matrix / doc_topic_matrix.sum(axis=1, keepdims=True)

# Calculate topic overlap between two documents
def topic_overlap(doc1_topics, doc2_topics):
    return np.sum(np.minimum(doc1_topics, doc2_topics))

# Build Graph with Adjusted Weights for Topic Overlap and Content Similarity
def build_graph(df, similarity_matrix, solved_questions, topic_matrix):
    G = nx.Graph()
    for idx, row in df.iterrows():
        G.add_node(row['titleSlug'], difficulty=row['difficulty'], likability=row['likability'], accuracy=row['accuracy'])
    
    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            prob = np.random.random()  # Mock for Bayesian Probability
            content_similarity = similarity_matrix[i, j]
            topic_overlap_score = topic_overlap(topic_matrix[i], topic_matrix[j])
            
            # Adjust edge weight based on both content similarity and topic overlap
            combined_weight = (content_similarity + topic_overlap_score) / 2
            G.add_edge(df.loc[i, 'titleSlug'], df.loc[j, 'titleSlug'], weight=combined_weight)
    
    return G

def recommend_questions(solved_questions, top_n=3, df=None):
    # If DataFrame is not passed, load it
    if df is None:
        df = load_data()

    question_vectors = TfidfVectorizer().fit_transform(df['question'])
    similarity_matrix = cosine_similarity(question_vectors)

    # Topic modeling (using custom topic model or any other topic model)
    topic_matrix = custom_topic_model(df['question'].tolist(), n_topics=3)

    # Dynamically calculate the potential matrix from data
    potential_matrix = calculate_potential_matrix(df)

    mrf = MarkovRandomField()
    mrf.add_edge('accuracy', 'difficulty', potential_matrix)

    # Perform belief propagation
    cpt = belief_propagation(mrf)

    # Build the graph with adjusted weights for similarity and topic overlap
    G = build_graph(df, similarity_matrix, solved_questions, topic_matrix)

    # Get the top N recommended questions
    recommendations = {}
    for solved in solved_questions:
        if solved not in G:
            continue
        neighbors = sorted(G[solved].items(), key=lambda x: x[1]['weight'], reverse=True)
        for neighbor, _ in neighbors:
            if neighbor not in solved_questions:
                recommendations[neighbor] = G[solved][neighbor]['weight']

    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]


In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

class QuestionRecommender:
    def __init__(self, file_path="updated_data.json"):
        self.df = self.load_and_preprocess_data(file_path)
        self.similarity_matrix = None
        self.topic_matrix = None
        self.potential_matrix = None
        self.mrf = None
        self.G = None

        # Pre-calculate matrices and graph once
        self.calculate_similarity_matrix()
        self.calculate_topic_matrix()
        self.calculate_potential_matrix()
        self.build_graph()

    # Load and preprocess DataFrame only once
    def load_and_preprocess_data(self, file_path):
        df = pd.read_json(file_path)
        df = self.preprocess_data(df)
        return df

    # Preprocess data (normalizing 'likability' and 'accuracy')
    def preprocess_data(self, df):
        scaler = MinMaxScaler()
        df[['likability', 'accuracy']] = scaler.fit_transform(df[['likability', 'accuracy']])
        return df

    # Function to calculate the similarity matrix using TF-IDF
    def calculate_similarity_matrix(self):
        question_vectors = TfidfVectorizer().fit_transform(self.df['question'])
        self.similarity_matrix = cosine_similarity(question_vectors)

    # Function to calculate the topic matrix (using custom topic model)
    def calculate_topic_matrix(self):
        self.topic_matrix = self.custom_topic_model(self.df['question'].tolist(), n_topics=3)

    # Function to calculate the joint probability of accuracy and difficulty (potential matrix)
    def calculate_potential_matrix(self):
        joint_prob = pd.crosstab(self.df['accuracy'], self.df['difficulty'], normalize='all')
        self.potential_matrix = np.zeros((2, 3))  # 2 values for accuracy (0, 1) and 3 values for difficulty (1, 2, 3)
        
        for acc in range(2):  # Accuracy can be 0 or 1
            for diff in range(1, 4):  # Difficulty can be 1, 2, or 3
                if diff in joint_prob.columns:
                    self.potential_matrix[acc, diff-1] = joint_prob.loc[acc, diff] if acc in joint_prob.index else 0

    # Markov Random Field (MRF) Implementation
    class MarkovRandomField:
        def __init__(self):
            self.edges = {}
            self.nodes = {}
            self.potentials = {}
        
        def add_edge(self, node1, node2, potential):
            if node1 not in self.nodes:
                self.nodes[node1] = {}
            if node2 not in self.nodes:
                self.nodes[node2] = {}
            
            self.potentials[(node1, node2)] = potential
        
        def compute_potential(self, node1, node2, val1, val2):
            return self.potentials[(node1, node2)][val1, val2]

    # Belief Propagation Implementation
    def belief_propagation(self, mrf, max_iter=10):
        messages = {}
        
        # Initialize messages to ones
        for (node1, node2) in mrf.edges:
            messages[(node1, node2)] = np.ones(2)  # Uniform initialization for binary values (0, 1)
        
        for _ in range(max_iter):
            for (node1, node2) in mrf.edges:
                incoming_messages = np.prod([messages[(nbr, node1)] for nbr in mrf.nodes[node1] if nbr != node2], axis=0)
                
                for val1 in range(2):  # For accuracy, which can take values 0 or 1
                    for val2 in range(3):  # For difficulty, which can take values 1, 2, or 3
                        messages[(node1, node2)] = mrf.compute_potential(node1, node2, val1, val2)
        
        return messages

    # Custom Topic Modeling (similar to LDA)
    def custom_topic_model(self, corpus, n_topics=2, alpha=0.1, beta=0.01):
        vocab = {word: idx for idx, word in enumerate(set(' '.join(corpus).split()))}
        word_topic_matrix = np.zeros((len(vocab), n_topics))
        doc_topic_matrix = np.zeros((len(corpus), n_topics))
        
        doc_word_topic = []
        for i, doc in enumerate(corpus):
            topics = []
            for word in doc.split():
                topic = np.random.randint(0, n_topics)
                topics.append(topic)
                word_topic_matrix[vocab[word], topic] += 1
                doc_topic_matrix[i, topic] += 1
            doc_word_topic.append(topics)
        
        for _ in range(50):
            for d, doc in enumerate(corpus):
                for i, word in enumerate(doc.split()):
                    topic = doc_word_topic[d][i]
                    word_topic_matrix[vocab[word], topic] -= 1
                    doc_topic_matrix[d, topic] -= 1
                    
                    topic_dist = (word_topic_matrix[vocab[word], :] + beta) * (doc_topic_matrix[d, :] + alpha)
                    new_topic = np.argmax(topic_dist / topic_dist.sum())
                    
                    doc_word_topic[d][i] = new_topic
                    word_topic_matrix[vocab[word], new_topic] += 1
                    doc_topic_matrix[d, new_topic] += 1
        
        return doc_topic_matrix / doc_topic_matrix.sum(axis=1, keepdims=True)

    # Calculate topic overlap between two documents
    def topic_overlap(self, doc1_topics, doc2_topics):
        return np.sum(np.minimum(doc1_topics, doc2_topics))

    # Build Graph with Adjusted Weights for Topic Overlap and Content Similarity
    def build_graph(self):
        self.G = nx.Graph()
        for idx, row in self.df.iterrows():
            self.G.add_node(row['titleSlug'], difficulty=row['difficulty'], likability=row['likability'], accuracy=row['accuracy'])
        
        for i in range(len(self.df)):
            for j in range(i + 1, len(self.df)):
                prob = np.random.random()  # Mock for Bayesian Probability
                content_similarity = self.similarity_matrix[i, j]
                topic_overlap_score = self.topic_overlap(self.topic_matrix[i], self.topic_matrix[j])
                
                combined_weight = (content_similarity + topic_overlap_score) / 2
                self.G.add_edge(self.df.loc[i, 'titleSlug'], self.df.loc[j, 'titleSlug'], weight=combined_weight)

    # Function to recommend questions based on solved questions
    def recommend_questions(self, solved_questions, top_n=3):
        recommendations = {}
        for solved in solved_questions:
            if solved not in self.G:
                continue
            neighbors = sorted(self.G[solved].items(), key=lambda x: x[1]['weight'], reverse=True)
            for neighbor, _ in neighbors:
                if neighbor not in solved_questions:
                    recommendations[neighbor] = self.G[solved][neighbor]['weight']

        return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Example usage:
recommender = QuestionRecommender()  # Load and preprocess once
solved_questions = ["two-sum", "add-two-numbers"]
top_n = 3
recommended = recommender.recommend_questions(solved_questions, top_n)
print("Recommendations:", recommended)


Recommendations: [('add-two-numbers-ii', 0.9366081613936407), ('merge-nodes-in-between-zeros', 0.7158952687162674), ('convert-binary-number-in-a-linked-list-to-integer', 0.7112255347854736)]


In [3]:
# Example usage:
solved_questions = ["integer-to-roman", "same-tree"]
top_n = 10
recommended = recommender.recommend_questions(solved_questions, top_n)
print("Recommendations:", recommended)

Recommendations: [('find-bottom-left-tree-value', 0.7848700913010869), ('unique-binary-search-trees', 0.750924606009741), ('balanced-binary-tree', 0.7479847047715535), ('find-duplicate-subtrees', 0.7471498169598703), ('merge-bsts-to-create-single-bst', 0.742378234003862), ('validate-binary-search-tree', 0.7362942750998073), ('unique-binary-search-trees-ii', 0.7308563360277761), ('rotate-list', 0.7245097755348765), ('sort-list', 0.723826635932493), ('two-sum-iv-input-is-a-bst', 0.7232214571714125)]


## **User Skill Level & Tine Out**

In [26]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

class QuestionRecommender:
    def __init__(self, file_path="updated_data.json"):
        self.df = self.load_and_preprocess_data(file_path)
        self.similarity_matrix = None
        self.topic_matrix = None
        self.potential_matrix = None
        self.mrf = None
        self.G = None
        self.user_skill_levels = {}  # Store skill levels for users
        self.user_solved_problems = {}  # Track solved problems by user
        self.timeout_problems = {}  # Problems that are 'timed out' for the user

        # Pre-calculate matrices and graph once
        self.calculate_similarity_matrix()
        self.calculate_topic_matrix()
        self.calculate_potential_matrix()
        self.build_graph()

    def load_and_preprocess_data(self, file_path):
        df = pd.read_json(file_path)
        df = self.preprocess_data(df)
        return df

    def preprocess_data(self, df):
        scaler = MinMaxScaler()
        df[['likability', 'accuracy']] = scaler.fit_transform(df[['likability', 'accuracy']])
        return df

    def calculate_similarity_matrix(self):
        question_vectors = TfidfVectorizer().fit_transform(self.df['question'])
        self.similarity_matrix = cosine_similarity(question_vectors)

    def calculate_topic_matrix(self):
        self.topic_matrix = self.custom_topic_model(self.df['question'].tolist(), n_topics=3)

    def calculate_potential_matrix(self):
        joint_prob = pd.crosstab(self.df['accuracy'], self.df['difficulty'], normalize='all')
        self.potential_matrix = np.zeros((2, 3))
        
        for acc in range(2):
            for diff in range(1, 4):
                if diff in joint_prob.columns:
                    self.potential_matrix[acc, diff-1] = joint_prob.loc[acc, diff] if acc in joint_prob.index else 0

    class MarkovRandomField:
        def __init__(self):
            self.edges = {}
            self.nodes = {}
            self.potentials = {}
        
        def add_edge(self, node1, node2, potential):
            if node1 not in self.nodes:
                self.nodes[node1] = {}
            if node2 not in self.nodes:
                self.nodes[node2] = {}
            
            self.potentials[(node1, node2)] = potential
        
        def compute_potential(self, node1, node2, val1, val2):
            return self.potentials[(node1, node2)][val1, val2]

    def belief_propagation(self, mrf, max_iter=10):
        messages = {}
        
        for (node1, node2) in mrf.edges:
            messages[(node1, node2)] = np.ones(2)  # Uniform initialization for binary values (0, 1)
        
        for _ in range(max_iter):
            for (node1, node2) in mrf.edges:
                incoming_messages = np.prod([messages[(nbr, node1)] for nbr in mrf.nodes[node1] if nbr != node2], axis=0)
                
                for val1 in range(2):
                    for val2 in range(3):
                        messages[(node1, node2)] = mrf.compute_potential(node1, node2, val1, val2)
        
        return messages

    def custom_topic_model(self, corpus, n_topics=2, alpha=0.1, beta=0.01):
        vocab = {word: idx for idx, word in enumerate(set(' '.join(corpus).split()))}
        word_topic_matrix = np.zeros((len(vocab), n_topics))
        doc_topic_matrix = np.zeros((len(corpus), n_topics))
        
        doc_word_topic = []
        for i, doc in enumerate(corpus):
            topics = []
            for word in doc.split():
                topic = np.random.randint(0, n_topics)
                topics.append(topic)
                word_topic_matrix[vocab[word], topic] += 1
                doc_topic_matrix[i, topic] += 1
            doc_word_topic.append(topics)
        
        for _ in range(50):
            for d, doc in enumerate(corpus):
                for i, word in enumerate(doc.split()):
                    topic = doc_word_topic[d][i]
                    word_topic_matrix[vocab[word], topic] -= 1
                    doc_topic_matrix[d, topic] -= 1
                    
                    topic_dist = (word_topic_matrix[vocab[word], :] + beta) * (doc_topic_matrix[d, :] + alpha)
                    new_topic = np.argmax(topic_dist / topic_dist.sum())
                    
                    doc_word_topic[d][i] = new_topic
                    word_topic_matrix[vocab[word], new_topic] += 1
                    doc_topic_matrix[d, new_topic] += 1
        
        return doc_topic_matrix / doc_topic_matrix.sum(axis=1, keepdims=True)

    def topic_overlap(self, doc1_topics, doc2_topics):
        return np.sum(np.minimum(doc1_topics, doc2_topics))

    def build_graph(self):
        self.G = nx.Graph()
        for idx, row in self.df.iterrows():
            self.G.add_node(row['titleSlug'], difficulty=row['difficulty'], likability=row['likability'], accuracy=row['accuracy'])
        
        for i in range(len(self.df)):
            for j in range(i + 1, len(self.df)):
                prob = np.random.random()  
                content_similarity = self.similarity_matrix[i, j]
                topic_overlap_score = self.topic_overlap(self.topic_matrix[i], self.topic_matrix[j])
                
                combined_weight = (content_similarity + topic_overlap_score) / 2
                self.G.add_edge(self.df.loc[i, 'titleSlug'], self.df.loc[j, 'titleSlug'], weight=combined_weight)

    def recommend_questions(self, user_id, solved_questions, top_n=3):
        # Initialize user skill level if not already set
        if user_id not in self.user_skill_levels:
            self.user_skill_levels[user_id] = 50  # Default skill level (1 to 100)
        
        recommendations = {}
        
        for solved in solved_questions:
            if solved not in self.G:
                continue
            neighbors = sorted(self.G[solved].items(), key=lambda x: x[1]['weight'], reverse=True)
            for neighbor, _ in neighbors:
                if neighbor not in solved_questions and neighbor not in self.timeout_problems.get(user_id, []):
                    problem_difficulty = self.df[self.df['titleSlug'] == neighbor]['difficulty'].values[0]
                    # Map difficulty to skill level (adjust as per requirements)
                    mapped_difficulty = self.difficulty_to_skill_level(problem_difficulty)
                    
                    # Only recommend problems that match user's current skill level
                    if mapped_difficulty <= self.user_skill_levels[user_id]:
                        recommendations[neighbor] = self.G[solved][neighbor]['weight']

        sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
        return sorted_recommendations[:top_n]

    def difficulty_to_skill_level(self, difficulty):
        # Map problem difficulty (1-5) to a skill level (1-100)
        skill_mapping = {1: 20, 2: 40, 3: 60, 4: 80, 5: 100}  # Customize as needed
        return skill_mapping.get(difficulty, 50)

    def update_skill_level(self, user_id, problem, solved):
        # Update user skill level after solving or timing out a problem based on difficulty
        problem_difficulty = self.df[self.df['titleSlug'] == problem]['difficulty'].values[0]
        mapped_difficulty = self.difficulty_to_skill_level(problem_difficulty)
        
        if solved:
            # If the user solved the problem, increase their skill level
            self.user_skill_levels[user_id] = min(self.user_skill_levels[user_id] + (mapped_difficulty // 10), 100)
        else:
            # Mark the problem as 'timed out' and don't recommend until the user reaches the required skill level
            if user_id not in self.timeout_problems:
                self.timeout_problems[user_id] = []
            self.timeout_problems[user_id].append(problem)

    def reset_timeout_problems(self, user_id):
        # Reset timeout problems for users who reach the highest skill level
        if self.user_skill_levels.get(user_id, 0) == 100:
            self.timeout_problems[user_id] = []


In [27]:
# Usage Example:
recommender = QuestionRecommender()

# Initialize user and solved questions
user_id = '1'
solved_questions = ["two-sum", "add-two-numbers"]
# Recommend questions based on the user's current skill level
recommendations = recommender.recommend_questions(user_id, solved_questions, 10)
print(recommendations)

[('add-two-numbers-ii', np.float64(0.9363931076301998)), ('convert-binary-number-in-a-linked-list-to-integer', np.float64(0.7192900509145057)), ('merge-nodes-in-between-zeros', np.float64(0.7191100249745019)), ('swapping-nodes-in-a-linked-list', np.float64(0.7114226741622482)), ('sort-list', np.float64(0.7110043002136168)), ('reverse-linked-list', np.float64(0.7102194657114697)), ('double-a-number-represented-as-a-linked-list', np.float64(0.7075104463777359)), ('remove-duplicates-from-sorted-list-ii', np.float64(0.7070358581962383)), ('remove-duplicates-from-sorted-list', np.float64(0.7016625457474788)), ('odd-even-linked-list', np.float64(0.7005874766742842))]


In [28]:
# Update skill level after solving a problem
recommender.update_skill_level(user_id, 'add-two-numbers-ii', solved=False)

In [29]:
# # Usage Example:
# recommender = QuestionRecommender()

# # Initialize user and solved questions
# user_id = '1'
solved_questions = ["two-sum", "add-two-numbers"]
# Recommend questions based on the user's current skill level
recommendations = recommender.recommend_questions(user_id, solved_questions, 10)
print(recommendations)

[('convert-binary-number-in-a-linked-list-to-integer', np.float64(0.7192900509145057)), ('merge-nodes-in-between-zeros', np.float64(0.7191100249745019)), ('swapping-nodes-in-a-linked-list', np.float64(0.7114226741622482)), ('sort-list', np.float64(0.7110043002136168)), ('reverse-linked-list', np.float64(0.7102194657114697)), ('double-a-number-represented-as-a-linked-list', np.float64(0.7075104463777359)), ('remove-duplicates-from-sorted-list-ii', np.float64(0.7070358581962383)), ('remove-duplicates-from-sorted-list', np.float64(0.7016625457474788)), ('odd-even-linked-list', np.float64(0.7005874766742842)), ('find-n-unique-integers-sum-up-to-zero', np.float64(0.6993016240310461))]


In [3]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import networkx as nx


class QuestionRecommender:
    def __init__(self, file_path="updated_data.json", skill_file="user_skill_levels.json"):
        self.df = self.load_and_preprocess_data(file_path)
        self.skill_file = skill_file
        self.similarity_matrix = None
        self.topic_matrix = None
        self.potential_matrix = None
        self.mrf = None
        self.G = None
        self.user_skill_levels = self.load_user_skill_levels()  # Load skill levels from JSON
        self.user_solved_problems = {}  # Track solved problems by user
        self.timeout_problems = {}  # Problems that are 'timed out' for the user

        # Pre-calculate matrices and graph once
        self.calculate_similarity_matrix()
        self.calculate_topic_matrix()
        self.calculate_potential_matrix()
        self.build_graph()

    def load_and_preprocess_data(self, file_path):
        df = pd.read_json(file_path)
        df = self.preprocess_data(df)
        return df

    def preprocess_data(self, df):
        scaler = MinMaxScaler()
        df[['likability', 'accuracy']] = scaler.fit_transform(df[['likability', 'accuracy']])
        return df

    def load_user_skill_levels(self):
        # Load the user skill levels from a JSON file
        try:
            with open(self.skill_file, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return {}  # Return empty dict if the file doesn't exist

    def save_user_skill_levels(self):
        # Save the user skill levels to the JSON file
        with open(self.skill_file, 'w') as f:
            json.dump(self.user_skill_levels, f, indent=4)

    def calculate_similarity_matrix(self):
        question_vectors = TfidfVectorizer().fit_transform(self.df['question'])
        self.similarity_matrix = cosine_similarity(question_vectors)

    def calculate_topic_matrix(self):
        self.topic_matrix = self.custom_topic_model(self.df['question'].tolist(), n_topics=3)

    def calculate_potential_matrix(self):
        joint_prob = pd.crosstab(self.df['accuracy'], self.df['difficulty'], normalize='all')
        self.potential_matrix = np.zeros((2, 3))

        for acc in range(2):
            for diff in range(1, 4):
                if diff in joint_prob.columns:
                    self.potential_matrix[acc, diff - 1] = joint_prob.loc[acc, diff] if acc in joint_prob.index else 0

    class MarkovRandomField:
        def __init__(self):
            self.edges = {}
            self.nodes = {}
            self.potentials = {}

        def add_edge(self, node1, node2, potential):
            if node1 not in self.nodes:
                self.nodes[node1] = {}
            if node2 not in self.nodes:
                self.nodes[node2] = {}

            self.potentials[(node1, node2)] = potential

        def compute_potential(self, node1, node2, val1, val2):
            return self.potentials[(node1, node2)][val1, val2]

    def belief_propagation(self, mrf, max_iter=10):
        messages = {}

        # Initialize messages
        for (node1, node2) in mrf.edges:
            messages[(node1, node2)] = np.ones(2)  # Uniform initialization for binary values (0, 1)

        for _ in range(max_iter):
            for (node1, node2) in mrf.edges:
                incoming_messages = np.prod([messages[(nbr, node1)] for nbr in mrf.nodes[node1] if nbr != node2], axis=0)

                for val1 in range(2):
                    for val2 in range(3):
                        messages[(node1, node2)] = mrf.compute_potential(node1, node2, val1, val2)

        return messages

    def custom_topic_model(self, corpus, n_topics=2, alpha=0.1, beta=0.01):
        vocab = {word: idx for idx, word in enumerate(set(' '.join(corpus).split()))}
        word_topic_matrix = np.zeros((len(vocab), n_topics))
        doc_topic_matrix = np.zeros((len(corpus), n_topics))

        doc_word_topic = []
        for i, doc in enumerate(corpus):
            topics = []
            for word in doc.split():
                topic = np.random.randint(0, n_topics)
                topics.append(topic)
                word_topic_matrix[vocab[word], topic] += 1
                doc_topic_matrix[i, topic] += 1
            doc_word_topic.append(topics)

        for _ in range(50):
            for d, doc in enumerate(corpus):
                for i, word in enumerate(doc.split()):
                    topic = doc_word_topic[d][i]
                    word_topic_matrix[vocab[word], topic] -= 1
                    doc_topic_matrix[d, topic] -= 1

                    topic_dist = (word_topic_matrix[vocab[word], :] + beta) * (doc_topic_matrix[d, :] + alpha)
                    new_topic = np.argmax(topic_dist / topic_dist.sum())

                    doc_word_topic[d][i] = new_topic
                    word_topic_matrix[vocab[word], new_topic] += 1
                    doc_topic_matrix[d, new_topic] += 1

        return doc_topic_matrix / doc_topic_matrix.sum(axis=1, keepdims=True)

    def topic_overlap(self, doc1_topics, doc2_topics):
        return np.sum(np.minimum(doc1_topics, doc2_topics))

    def build_graph(self):
        self.G = nx.Graph()
        for idx, row in self.df.iterrows():
            self.G.add_node(row['titleSlug'], difficulty=row['difficulty'], likability=row['likability'], accuracy=row['accuracy'])

        for i in range(len(self.df)):
            for j in range(i + 1, len(self.df)):
                prob = np.random.random()  
                content_similarity = self.similarity_matrix[i, j]
                topic_overlap_score = self.topic_overlap(self.topic_matrix[i], self.topic_matrix[j])

                combined_weight = (content_similarity + topic_overlap_score) / 2
                self.G.add_edge(self.df.loc[i, 'titleSlug'], self.df.loc[j, 'titleSlug'], weight=combined_weight)

    def recommend_questions(self, user_id, solved_questions, top_n=3):
        # Initialize user skill level if not already set
        if user_id not in self.user_skill_levels:
            self.user_skill_levels[user_id] = 1  # Starting skill level

        recommendations = {}

        for solved in solved_questions:
            if solved not in self.G:
                continue
            neighbors = sorted(self.G[solved].items(), key=lambda x: x[1]['weight'], reverse=True)
            for neighbor, _ in neighbors:
                if neighbor not in solved_questions and neighbor not in self.timeout_problems.get(user_id, []):
                    problem_difficulty = self.df[self.df['titleSlug'] == neighbor]['difficulty'].values[0]
                    # Map difficulty to skill level (adjust as per requirements)
                    mapped_difficulty = self.difficulty_to_skill_level(problem_difficulty)

                    # Only recommend problems that match user's current skill level
                    if mapped_difficulty <= self.user_skill_levels[user_id]:
                        recommendations[neighbor] = self.G[solved][neighbor]['weight']

        sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

        # Perform belief propagation to adjust the recommendations
        self.mrf = self.MarkovRandomField()
        self.mrf.add_edge('accuracy', 'difficulty', self.potential_matrix)

        # Update belief propagation for each recommendation
        cpt = self.belief_propagation(self.mrf)

        # Adjust recommendations using belief propagation output
        adjusted_recommendations = []
        for rec, _ in sorted_recommendations[:top_n]:
            # Modify recommendation score using belief propagation (CPT output)
            adjustment_factor = cpt.get(('accuracy', 'difficulty'), 1)
            adjusted_recommendations.append((rec, adjustment_factor))

        return adjusted_recommendations[:top_n]

    def difficulty_to_skill_level(self, difficulty):
        # Map problem difficulty (1-5) to a skill level (1-100)
        skill_mapping = {1: 20, 2: 40, 3: 60, 4: 80, 5: 100}  # Customize as needed
        return skill_mapping.get(difficulty, 50)

    def update_skill_level(self, user_id, problem, solved):
        # Update user skill level after solving or timing out a problem based on difficulty
        problem_difficulty = self.df[self.df['titleSlug'] == problem]['difficulty'].values[0]
        skill_increment = self.difficulty_to_skill_level(problem_difficulty)

        if solved:
            self.user_skill_levels[user_id] += skill_increment
        else:
            # If timed out, decrease skill slightly
            self.user_skill_levels[user_id] -= skill_increment / 2

        # Make the progression slower after certain levels
        self.user_skill_levels[user_id] = np.minimum(self.user_skill_levels[user_id], 100)
        self.user_skill_levels[user_id] = np.maximum(self.user_skill_levels[user_id], 1)

        # Save updated skill levels to the JSON file
        self.save_user_skill_levels()

In [4]:
file_path = "updated_data.json"
recommender = QuestionRecommender(file_path)

user_id = '1'
solved_questions = ["two-sum", "add-two-numbers"]
# Recommend questions based on the user's current skill level
recommendations = recommender.recommend_questions(user_id, solved_questions, 10)
print(recommendations)

[]


In [35]:
recommender.update_skill_level(user_id, 'add-two-numbers-ii', solved=False)

User 1's skill level updated to: 50


In [5]:
# # Usage Example:
# recommender = QuestionRecommender()

# # Initialize user and solved questions
# user_id = '1'
solved_questions = ["two-sum", "add-two-numbers"]
# Recommend questions based on the user's current skill level
recommendations = recommender.recommend_questions(user_id, solved_questions, 10)
print(recommendations)

[]
